# Install required packages

In [1]:
!pip install opencv-python

In [2]:
import numpy as np
import cv2

# Select which video to analize

Replace INPUT_VIDEO_PATH var below with the name of video you want to analyze

In [3]:
INPUT_VIDEO_PATH = "input_files/Traffic_Laramie_1.mp4"


# Utility functions

The below function extracts the background from a video and returns a frame with all moving objects removed. To do this it obtains the median over 50 random frames selected from the video.

In [4]:
def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    # we will randomly select 50 frames for the calculating the median
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)
    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    return median_frame

# Read the video

## Hyperparameters used for motion detection

Below we define the hyperparameters used by the algorithm.  
CONSECUTIVE_FRAMES is the number of consecutive frames we will analize together to finding object contours, incrementing it will reduce computation but drawing boxes will be less accurate.  
MINIMAL_SIZE: minimal area in motion to be detected, keep it high enough to avoid detecting pedestrians and cars that are far.

In [5]:
CONSECUTIVE_FRAMES = 2 # how manny frames will be compared to detect movement, less = more detailed detection, more computation needed
MINIMAL_SIZE = 2000 # minimal object size, big enough to detect cars but avoid pedestrians

## read video and instantiate video writer

Reads input video and extracts width, height and fps.
then uses those parameters to instantiate a video writer that will be used to write the updated frames to disk.

In [6]:
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
# get the video frame height and width
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

save_name = f"output_files_task1/{INPUT_VIDEO_PATH.split('/')[-1]}"
# define codec and create VideoWriter object
out = cv2.VideoWriter(
    save_name,
    cv2.VideoWriter_fourcc(*'mp4v'), fps, 
    (frame_width, frame_height)
)

In [7]:
fps

25

input video runs at 25 fps

# Get background frame

Applies background substraction technique to obtain a frame with moving objects removed

In [8]:
# get the background model
background = get_background(INPUT_VIDEO_PATH)
# convert the background model to grayscale format
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
frame_count = 0

# Loop over video frames and detect moving objects

Loop over input video frames and do the following:  
1- convert frames to grayscale for easier detection  
2- find difference between current frame and background frame, ie: remove the background.  
3- apply a threshold to convert all pixels to either 1 or 0 depending on difference with background.  
4- sum consecutive frames(with background removed): find contours around objects, represented as group of pixels clustered together with values above 0, only reason to do this is to reduce computation. finding the contour on a sequence of CONSECUTIVE_FRAMES frames is CONSECUTIVE_FRAMES times faster than doing it for individual frames.  
5- Use OpenCV's findContours function to find contours for consecutive pixels that have non 0 values. We only find contours for the last pixel of each group of CONSECUTIVE_FRAMES, since need the previous frames of the group to calculate it(Later I still use the same contour for every pixel in the group)  
6- append each frame with detected contours(if available) to a list of frames to later be written to disk.

In [9]:
frame_diff_list = []
frames_to_write = []
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        orig_frame = frame.copy()
        # IMPORTANT STEP: convert the frame to grayscale first
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if frame_count % CONSECUTIVE_FRAMES == 0 or frame_count == 1:
            frame_diff_list = []
        # find the difference between current frame and base frame
        frame_diff = cv2.absdiff(gray, background)
        # thresholding to convert the frame to binary
        ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
        # dilate the frame a bit to get some more white area...
        # ... makes the detection of contours a bit easier
        dilate_frame = cv2.dilate(thres, None, iterations=2)
        # append the final result into the `frame_diff_list`
        frame_diff_list.append(dilate_frame)
        # if we have reached `CONSECUTIVE_FRAMES` number of frames
        if len(frame_diff_list) == CONSECUTIVE_FRAMES:
            # add all the frames in the `frame_diff_list`
            sum_frames = sum(frame_diff_list)
            # find the contours around the white segmented areas
            contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            frames_to_write.append((orig_frame, contours))
        else:
            # write intermediate frames to avoid frame skipping in output video? 
            frames_to_write.append((orig_frame, []))
    else:
        break


# write frames with the contours to disk

Loop over frames, contours list processed in previous step and for each frame:
1) If don't have contours for this image, find the next contour in the list
2) draw all contours as rectangles in the frame
2) Write frame to disk

In [10]:
i=0
for frame, contours in frames_to_write:
    j=1
    while(not contours and (i+j) <len(frames_to_write)):
        contours = frames_to_write[i+j][1]
        j+=1
    i+=1
    for contour in contours:
        # continue through the loop if contour area is less than MINIMAL_SIZE...
        # ... helps in removing noise detection
        if cv2.contourArea(contour) < MINIMAL_SIZE:
            continue
        # get the xmin, ymin, width, and height coordinates from the contours
        (x, y, w, h) = cv2.boundingRect(contour)
        # draw the bounding boxes
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    out.write(frame)
    
cap.release()
out.release()

# Sources

Some of the ideas I used to implement my application are based on the following articles:
- https://research.ijcaonline.org/volume102/number7/pxc3898647.pdf
- https://debuggercafe.com/moving-object-detection-using-frame-differencing-with-opencv/